In [9]:
import sys
import numpy as np
import pandas as pd
import json
 
sys.path.append('../')

from functions import *

/Users/liumukun/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [10]:
data = pd.read_csv("../../data/demeaned.csv")
df = data.groupby(["id", "ispolice", "sellerfeedbackscore", "bidcount", "apple", "amazon", "increment_residual"])["residual"].apply(lambda x: x.values).reset_index()

valid_bids = list(df[df.ispolice == 1].bidcount.value_counts().index)
include = df[(df.bidcount > 1) & (df.bidcount.isin(valid_bids))]

bids = list(include.residual)

logged_feedback = np.log(include.sellerfeedbackscore+1)
logged_feedback = transform_covariates(logged_feedback, 100)
include.sellerfeedbackscore = logged_feedback

covariates = np.array(include[["ispolice", "sellerfeedbackscore"]])
covariates = list([list(cov) for cov in covariates])
incremented = list(include.increment_residual)

<ipython-input-10-3210d6cab888>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  include.sellerfeedbackscore = logged_feedback


In [5]:
with open("../dumps/CMF_main.json", "r") as f:
    d = json.load(f)
    median_lower = d["lower"]
    median_upper = d["upper"]

In [11]:
percentile = lambda p: np.percentile(np.array(include.sellerfeedbackscore), p)

cef1 = lambda c, cov: c[0]*(1-cov[0])+c[1]*cov[0]+c[2]*cov[1]*(1-cov[0])+c[3]*cov[1]*cov[0]
n_cov1 = 4

def cef2(c, cov):
    low = int(cov[1] >= percentile(0) and cov[1] < percentile(91.5))
    high = int(cov[1] >= percentile(91.5) and cov[1] <= percentile(100))
    return c[0]*cov[1] + c[1]*(1-cov[1]) + c[2]*cov[1]*high + c[3]*(1-cov[1])*high
n_cov2 = 4

def cef3(c,cov):
    feedback = np.exp(cov[1])-1
    return c[0]*(1-cov[0]) + c[1]*cov[0] + c[2]*feedback*(1-cov[0]) + c[3]*feedback*cov[0] + \
        c[4]*feedback**2*(1-cov[0]) + c[5]*feedback**2*cov[0]
n_cov3 = 6

def cef4(c,cov):
    feedback = np.exp(cov[1])-1
    return c[0]*(1-cov[0]) + c[1]*cov[0] + c[2]*feedback*(1-cov[0]) + c[3]*feedback*cov[0] + \
        c[4]*feedback**2*(1-cov[0]) + c[5]*feedback**2*cov[0] + c[6]*feedback**3*(1-cov[0]) + \
        c[7]*feedback**3*cov[0]
n_cov3 = 8

In [ ]:
def loss_function2(c):
    cef = lambda cov: cef2(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point2, interval2 = get_estimates(loss_function2, n_cov2)

print(point2)
print(interval2)

In [ ]:
def loss_function3(c):
    cef = lambda cov: cef3(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point3, interval3 = get_estimates(loss_function3, n_cov3)

print(point3)
print(interval3)

In [ ]:
def loss_function4(c):
    cef = lambda cov: cef3(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point4, interval4 = get_estimates(loss_function4, n_cov4)

print(point4)
print(interval4)